In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("02_Test_Data_Set/galaxy_users.csv")

In [4]:
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No


In [5]:
df.loc[:2, "OnlineSecurity":"DeviceProtection"]

,OnlineSecurity,OnlineBackup,DeviceProtection
0,No,Yes,No
1,Yes,No,Yes
2,Yes,Yes,No


In [12]:
df.iloc[0, :].reset_index()

,index,0
0,customerID,7590-VHVEG
1,gender,Female
2,SeniorCitizen,0
3,Partner,Yes
4,Dependents,No
5,tenure,1
6,PhoneService,No
7,MultipleLines,No phone service
8,InternetService,DSL
9,OnlineSecurity,No


In [25]:
df.iloc[:3, :5]

,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No


### Q1. 

In [13]:
# 인스턴스 하나에 대해서 할 때는 []로 감싸서 array 형태로 넣어야함
df.loc[[0], "OnlineSecurity":"StreamingMovies"]

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No


In [15]:
df.loc[[0], "OnlineSecurity":"StreamingMovies"].apply(lambda x: sum(x == "Yes"),
                                                      axis = 1)

0    1
dtype: int64

In [27]:
df_q1 = df.loc[:, "OnlineSecurity":"StreamingMovies"].reset_index(drop = True)
df_q1.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No
1,Yes,No,Yes,No,No,No


In [28]:
for n in range(6):
    print(df_q1.columns[n], df_q1.iloc[:, n].unique())

OnlineSecurity ['No' 'Yes' 'No internet service']
OnlineBackup ['Yes' 'No' 'No internet service']
DeviceProtection ['No' 'Yes' 'No internet service']
TechSupport ['No' 'Yes' 'No internet service']
StreamingTV ['No' 'Yes' 'No internet service']
StreamingMovies ['No' 'Yes' 'No internet service']


In [30]:
# df_q1.drop_duplicates()
df_q1.apply(lambda x: x.unique())
# 변수별 고유 원소 개수가 같으면 결과는 데이터프레임

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No
1,Yes,No,Yes,Yes,Yes,Yes
2,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service


In [25]:
df_q1_sub = df_q1.loc[df_q1["OnlineSecurity"] != "No internet service", ]
df_q1_sub.apply(lambda x: x.unique())
# ??? 다른 변수에 있는 것도 사라짐. --> 특정 row에 다 같이 있었기 때문

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No
1,Yes,No,Yes,Yes,Yes,Yes


In [27]:
# df_q1.loc[df_q1["OnlineSecurity"] == "No internet service", ]

In [30]:
df_q1_na = df_q1.replace("No internet service", np.nan)
# df_q1_na.isna().sum()
df_q1_notna = df_q1_na.dropna()
df_q1_notna.isna().sum()

OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64

In [31]:
df_q1_sub.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,No,Yes,No,No,No,No
1,Yes,No,Yes,No,No,No


In [33]:
df_q1_sub["cnt"] = df_q1_sub.apply(lambda x: sum(x == "Yes"), axis = 1)
df_q1_sub.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,cnt
0,No,Yes,No,No,No,No,1
1,Yes,No,Yes,No,No,No,2


In [35]:
srv_cnt = df_q1_sub["cnt"].value_counts()
srv_cnt

3    1117
2    1033
1     966
4     850
0     693
5     569
6     284
Name: cnt, dtype: int64

In [39]:
966 / 284

3.4014084507042255

In [38]:
round(srv_cnt[1] / srv_cnt[6], 1)

3.4

### Q2.

In [45]:
13 // 5 , 13 % 5 # 몫/나머지

(2, 3)

In [32]:
df_q2 = df[["tenure", "MonthlyCharges", "TotalCharges"]].reset_index(drop = True)
df_q2.head(2)

,tenure,MonthlyCharges,TotalCharges
0,1,29.85,29.85
1,34,56.95,1889.50


In [33]:
df_q2["m_cnt"] = df_q2["TotalCharges"] // df_q2["MonthlyCharges"]
df_q2.head(2)

,tenure,MonthlyCharges,TotalCharges,m_cnt
0,1,29.85,29.85,1.0
1,34,56.95,1889.50,33.0


In [34]:
df_q2.drop("TotalCharges", axis = 1).corr()

,tenure,MonthlyCharges,m_cnt
tenure,1.000000,0.246862,0.998831
MonthlyCharges,0.246862,1.000000,0.246164
m_cnt,0.998831,0.246164,1.000000


In [37]:
mat_corr = df_q2.drop("TotalCharges", axis = 1).corr()
mat_corr = mat_corr.reset_index()
mat_corr
mat_corr_melt = mat_corr.melt(id_vars = "index")
mat_corr_melt = mat_corr_melt.loc[mat_corr_melt["index"] != mat_corr_melt["variable"], ]
mat_corr_melt

,index,variable,value
1,MonthlyCharges,tenure,0.246862
2,m_cnt,tenure,0.998831
3,tenure,MonthlyCharges,0.246862
5,m_cnt,MonthlyCharges,0.246164
6,tenure,m_cnt,0.998831
7,MonthlyCharges,m_cnt,0.246164


In [39]:
mat_corr_melt.loc[[mat_corr_melt["value"].abs().idxmax()], ]

,index,variable,value
2,m_cnt,tenure,0.998831


### Q3.

In [40]:
df.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [41]:
df_q3 = df[["Churn", 
            'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'MonthlyCharges', 'TotalCharges', 
            'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingMovies', 'PaperlessBilling']]
df_q3 = df_q3.reset_index(drop = True)
df_q3.head(2)

,Churn,SeniorCitizen,Partner,Dependents,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,PaperlessBilling
0,No,0,Yes,No,1,29.85,29.85,No,Yes,No,No,No,Yes
1,No,0,No,No,34,56.95,1889.50,Yes,No,Yes,No,No,No


In [42]:
df_q3 = df_q3.replace(dict(Yes = 1, No = 0))
df_q3.head(2)

,Churn,SeniorCitizen,Partner,Dependents,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,PaperlessBilling
0,0,0,1,0,1,29.85,29.85,0,1,0,0,0,1
1,0,0,0,0,34,56.95,1889.50,1,0,1,0,0,0


In [43]:
df_q3.dtypes

Churn                 int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
MonthlyCharges      float64
TotalCharges        float64
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingMovies      object
PaperlessBilling      int64
dtype: object

In [51]:
df_q3_obj = df_q3.select_dtypes(exclude = "number")
df_q3_obj.head(2)

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies
0,0,1,0,0,0
1,1,0,1,0,0


In [64]:
df_q3_obj.apply(lambda x: x.unique())

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies
0,0,1,0,0,0
1,1,0,1,1,1
2,No internet service,No internet service,No internet service,No internet service,No internet service


In [66]:
df_q3 = df_q3.replace("No internet service", -1)

In [67]:
df_q3.dtypes

Churn                 int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
MonthlyCharges      float64
TotalCharges        float64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingMovies       int64
PaperlessBilling      int64
dtype: object

In [68]:
df_train, df_test = train_test_split(df_q3, train_size = 0.7,
                                     random_state = 123)

In [69]:
len(df_train), len(df_test)

(4922, 2110)

In [71]:
model_nor = MinMaxScaler()
model_nor.fit(df_train)
df_train_nor = model_nor.transform(df_train)
df_test_nor  = model_nor.transform(df_test)

In [73]:
df_train_nor[:1, ]

array([[1.        , 0.        , 0.        , 0.        , 0.08450704,
        0.81116094, 0.07551927, 0.5       , 1.        , 0.5       ,
        0.5       , 1.        , 1.        ]])

In [72]:
df_train.head(1)

,Churn,SeniorCitizen,Partner,Dependents,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,PaperlessBilling
463,1,0,0,0,7,99.8,673.25,0,1,0,0,1,1


In [75]:
model_lr = LogisticRegression(random_state = 123)
model_lr.fit(X = df_train_nor[:, 1:],
             y = df_train_nor[:, 0])

LogisticRegression(random_state=123)

In [81]:
pred = model_lr.predict(df_test_nor[:, 1:])
pred[:5]

In [79]:
round(f1_score(y_true = df_test_nor[:, 0],
               y_pred = pred), 2)

0.55

In [80]:
df_train_nor2 = pd.DataFrame(df_train_nor, 
                             columns = df_train.columns)
df_train_nor2.head(2)

,Churn,SeniorCitizen,Partner,Dependents,tenure,MonthlyCharges,TotalCharges,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingMovies,PaperlessBilling
0,1.0,0.0,0.0,0.0,0.084507,0.811161,0.075519,0.5,1.0,0.5,0.5,1.0,1.0
1,1.0,0.0,1.0,0.0,0.000000,0.607374,0.006987,0.5,0.5,0.5,0.5,0.5,1.0


범주형 변수 한 번에 처리.

In [82]:
df2 = pd.read_csv("02_Test_Data_Set/galaxy_users.csv")
df2.head(1)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No


In [85]:
df2_sub = df2.loc[:, "Partner":"OnlineSecurity"]
df2_sub.head(2)

,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,Yes,No,1,No,No phone service,DSL,No
1,No,No,34,Yes,No,DSL,Yes


In [86]:
df2_sub_obj = df2_sub.select_dtypes(exclude = "number")
df2_sub_obj.head(1)

,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,Yes,No,No,No phone service,DSL,No


In [88]:
df2_sub_obj_melt = df2_sub_obj.melt()
df2_sub_obj_melt.head(2)

,variable,value
0,Partner,Yes
1,Partner,No


In [89]:
df2_sub_obj_melt["value"].unique()

array(['Yes', 'No', 'No phone service', 'DSL', 'Fiber optic',
       'No internet service'], dtype=object)

In [94]:
ser_rep = pd.Series([1, 0, -1, -1, -1, -1],
                    index = df2_sub_obj_melt["value"].unique())
ser_rep

Yes                    1
No                     0
No phone service      -1
DSL                   -1
Fiber optic           -1
No internet service   -1
dtype: int64

In [95]:
df2_sub.head(2)

,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,Yes,No,1,No,No phone service,DSL,No
1,No,No,34,Yes,No,DSL,Yes


In [97]:
df2_sub = df2_sub.replace(ser_rep)
df2_sub.head()

,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity
0,1,0,1,0,-1,-1,0
1,0,0,34,1,0,-1,1
2,0,0,2,1,0,-1,1
3,0,0,45,0,-1,-1,1
4,0,0,2,1,0,-1,0
